In [1]:
import pandas as pd
import pyodbc
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.DataFrame(pd.read_csv('vin_request.csv', delimiter=';'))

In [4]:
df.head()

,Year,WMI,Brand,ModelName,distinct_users,distinct_requests
0,2015,101,NOT FOUND,NOT FOUND,1,1
1,2015,142,NOT FOUND,NOT FOUND,1,1
2,2015,19X,HONDA,CIVIC,1,1
3,2015,1B3,CHRYSLER,JA - BREEZE/ STRATUS/ CIRRUS,1,1
4,2015,1B3,CHRYSLER,JR - STRATUS/ SEBRING,5,12


In [5]:
train_df = pd.DataFrame(pd.read_csv('all_vin.csv', delimiter=';'))

In [23]:
train_df.rename(columns={'Бренд': 'Brand', 'Модель': 'ModelName'}, inplace=True)

In [24]:
train_df

,Brand,ModelName,WMI
0,ACURA,NaN,MRA
1,ACURA,NaN,2HH
2,ACURA,NaN,19U
3,ACURA,NaN,JH4
4,ACURA,NaN,5J8
...,...,...,...
71021,РАФ,3311,БН
71022,РАФ,4530,255
71023,ТРЭКОЛ,39041,419
71024,ТРЭКОЛ,39284,353


In [7]:
# Filter out rows where 'Brand' is 'Not Found'
missing_df_filtered = df[df['Brand'] == 'NOT FOUND']

# Select features for prediction
features_for_prediction = missing_df_filtered[['ModelName', 'WMI']]
print(features_for_prediction.head())

    ModelName  WMI
0   NOT FOUND  101
1   NOT FOUND  142
8   NOT FOUND  1B3
22  NOT FOUND  1c4
29  NOT FOUND  1FA


In [38]:
df.head()

Year  distinct_users  \
WMI Brand     ModelName                                            
101 NOT FOUND NOT FOUND                     2015               1   
142 NOT FOUND NOT FOUND                     2015               1   
19X HONDA     CIVIC                         2015               1   
1B3 CHRYSLER  JA - BREEZE/ STRATUS/ CIRRUS  2015               1   
              JR - STRATUS/ SEBRING         2015               5   

                                            distinct_requests  
WMI Brand     ModelName                                        
101 NOT FOUND NOT FOUND                                     1  
142 NOT FOUND NOT FOUND                                     1  
19X HONDA     CIVIC                                         1  
1B3 CHRYSLER  JA - BREEZE/ STRATUS/ CIRRUS                  1  
              JR - STRATUS/ SEBRING                        12

In [36]:
# Assuming 'df' and 'train_df' are your datasets
# Assuming 'WMI', 'Brand', 'ModelName' are the corresponding column names in both datasets

# Set the 'WMI', 'Brand', 'ModelName' columns as the index for both datasets
df.set_index(['WMI', 'Brand', 'ModelName'], inplace=True)
train_df.set_index(['WMI', 'Brand', 'ModelName'], inplace=True)

# Join datasets based on their indices with suffixes
joined_data = df.join(train_df, how='left', lsuffix='_left', rsuffix='_train')

# Replace 'NOT FOUND' with NaN for easier handling
joined_data.replace('NOT FOUND', pd.NA, inplace=True)

# Display the relevant columns in the joined data
relevant_columns = ['WMI', 'Brand', 'ModelName', 'distinct_users', 'distinct_requests', 'Brand_train', 'ModelName_train']
print(joined_data[relevant_columns].reset_index().head())


KeyError: "None of ['WMI', 'Brand', 'ModelName'] are in the columns"

In [29]:
# Replace 'NOT FOUND' with NaN for easier handling
joined_data.replace('NOT FOUND', pd.NA, inplace=True)

# Display the relevant columns in the joined data
relevant_columns = ['WMI', 'Brand', 'ModelName', 'distinct_users', 'distinct_requests', 'Brand_right', 'ModelName_right']
print(joined_data[relevant_columns].head())

KeyError: "['Brand_right', 'ModelName_right'] not in index"

In [22]:
# Replace 'NOT FOUND' with NaN for easier handling
merged_data.replace('NOT FOUND', pd.NA, inplace=True)

# Separate data with missing values and data with known values
known_data = merged_data.dropna(subset=['Brand', 'ModelName'])
missing_data = merged_data[merged_data['Brand'].isna() | merged_data['ModelName'].isna()]
known_data

,Year,WMI,Brand,ModelName,distinct_users,distinct_requests,Бренд,Модель
202,2015,19X,HONDA,CIVIC,1,1,HONDA,CIVIC
203,2015,1B3,CHRYSLER,JA - BREEZE/ STRATUS/ CIRRUS,1,1,CHRYSLER,NEON
204,2015,1B3,CHRYSLER,JA - BREEZE/ STRATUS/ CIRRUS,1,1,CHRYSLER,NaN
205,2015,1B3,CHRYSLER,JA - BREEZE/ STRATUS/ CIRRUS,1,1,CHRYSLER,STRATUS
206,2015,1B3,CHRYSLER,JA - BREEZE/ STRATUS/ CIRRUS,1,1,DODGE,NEON
...,...,...,...,...,...,...,...,...
1668376,2023,ZFA,FIAT PROFESSIONAL,NUOVO DUCATO [NUOVO DUCATO MY 2011 (2011-2014)],4,4,LMC,NaN
1668377,2023,ZFA,FIAT PROFESSIONAL,NUOVO DUCATO [NUOVO DUCATO MY 2011 (2011-2014)],4,4,ПРОЧИЕ,ПРОЧИЕ
1668378,2023,zfa,FIAT PROFESSIONAL,SCUDO [SCUDO (1995-2004)],3,3,NaN,NaN
1668496,2023,zla,LANCIA,DEDRA [DEDRA BENZINA (1989-1994)],3,4,NaN,NaN


In [18]:
from sklearn.model_selection import train_test_split

# Assuming 'merged_data' is your complete dataset
train_data, test_data = train_test_split(merged_data, test_size=0.2, random_state=42)


In [21]:
from sklearn.preprocessing import OneHotEncoder

# One-hot encode the 'WMI' column
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
train_encoded = pd.DataFrame(encoder.fit_transform(train_data[['WMI']]), columns=encoder.get_feature_names_out(['WMI']))
test_encoded = pd.DataFrame(encoder.transform(test_data[['WMI']]), columns=encoder.get_feature_names_out(['WMI']))

# Concatenate the encoded columns back to the original data
train_data = pd.concat([train_data, train_encoded], axis=1)
test_data = pd.concat([test_data, test_encoded], axis=1)

C:\Users\a.talgatkyzy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


MemoryError: Unable to allocate 107. GiB for an array with shape (1334828, 10721) and data type float64

In [19]:
from sklearn.ensemble import RandomForestClassifier

features = 'WMI'
target_brand = 'Brand'
target_model = 'ModelName'

model_brand = RandomForestClassifier(random_state=42)
model_model = RandomForestClassifier(random_state=42)

model_brand.fit(train_data[features], train_data[target_brand])
model_model.fit(train_data[features], train_data[target_model])



ValueError: could not convert string to float: 'WDB'

In [ ]:
predictions_brand = model_brand.predict(test_data[features])
predictions_model = model_model.predict(test_data[features])

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_brand = accuracy_score(test_data[target_brand], predictions_brand)
accuracy_model = accuracy_score(test_data[target_model], predictions_model)

print(f'Accuracy for Brand: {accuracy_brand:.2f}')
print(f'Accuracy for ModelName: {accuracy_model:.2f}')